# **1. Download and unzip the data** 💾
In case of errors, comment the %%capture magic command in the following cell

In [ ]:
%%capture
!pip install gdown xarray[complete]
from google.colab import drive
drive.mount('/content/drive')

file_id = '10yslT_c6jQN7FmrRXvRXjWFTju75iWa5'
output_path = '/content/drive/My Drive/Dataset.zip'

!gdown --id $file_id -O "$output_path"

In [ ]:
!unzip -o "$output_path" -d /content/drive/My\ Drive/Dataset/

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/My Drive/Dataset/lc_imgs/6.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/51.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/52.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/53.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/54.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/55.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/56.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/57.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/58.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/59.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/60.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/63.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/505.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/506.nc  
  inflating: /content/drive/My Drive/Dataset/lc_imgs/507.nc  
  inflating: /content/drive/My D

The dataset folder contains
1. Folder `lc_imgs` - This folder contains 10000 `netcdf` files containing high resolution satellite imagery.
2. `class_dict.json` - This file contains the mapping of segmentation values for the landcover maps

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!apt-get install tree
!tree /content/drive/My\ Drive/Dataset --filelimit=5

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tree is already the newest version (2.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
/content/drive/My Drive/Dataset
├── class_dict.json
└── lc_imgs  [10000 entries exceeds filelimit, not opening dir]

1 directory, 1 file


# **Model training + Evaluation**.


In [ ]:
import numpy as np
import json
import os
import xarray as xr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from sklearn.decomposition import PCA
from google.colab import drive
drive.mount('/content/drive')

# Load data from .nc files
data_dir = "/content/drive/My Drive/Dataset/"
nc_files = os.listdir(os.path.join(data_dir, 'lc_imgs'))
nc_files = nc_files[:4000]  # Selecting the first 4000 files

# Define the number of PCA components
num_components = 1

images = []
labels = []

for filename in nc_files:
    file_path = os.path.join(data_dir, 'lc_imgs', filename)
    ds = xr.open_dataset(file_path)
    blue = ds['b'].values
    green = ds['g'].values
    red = ds['r'].values
    nir = ds['nir'].values
    lc = ds['lc'].values

    red[np.isnan(red)] = 0
    green[np.isnan(green)] = 0
    blue[np.isnan(blue)] = 0
    nir[np.isnan(nir)] = 0

    # Stack bands to create image
    image = np.stack((red, green, blue, nir), axis=-1)

    # Flatten the image for PCA
    flattened_image = image.reshape(-1, image.shape[-1])

    # Apply PCA
    pca = PCA(n_components=num_components)
    pca_result = pca.fit_transform(flattened_image)

    # Reshape PCA result back to original shape
    pca_result_reshaped = pca_result.reshape(image.shape[0], image.shape[1], num_components)

    image = np.stack((blue, green, red, nir, pca_result_reshaped[:, :, 0]), axis=-1)
    # Append PCA result to images list
    images.append(image)

    # Append labels
    labels.append(lc)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Print the shape of images and labels
print("Image Shape with PCA:", images.shape)
print("Label Shape with PCA:", labels.shape)

unique_classes = set()
for label in labels:
    unique_classes.update(np.unique(label))

num_classes = len(unique_classes)
print("Number of classes:", num_classes)

# Split data into train, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(images, labels, test_size=0.4, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

print("Train set:")
print("  Image shape:", X_train.shape)
print("  Label shape:", y_train.shape)
print("Validation set:")
print("  Image shape:", X_val.shape)
print("  Label shape:", y_val.shape)
print("Test set:")
print("  Image shape:", X_test.shape)
print("  Label shape:", y_test.shape)
print("*************************************************************")
## Add normalization to input data
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0
print('After Normalization')
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

# Function to compute class distribution
def compute_class_distribution(labels, num_classes):
    class_counts = np.zeros(num_classes, dtype=int)
    total_pixels = 0

    for label in labels:
        unique_classes, class_counts_per_sample = np.unique(label, return_counts=True)
        unique_classes = unique_classes.astype(int)
        class_counts[unique_classes] += class_counts_per_sample
        total_pixels += np.sum(class_counts_per_sample)
    class_imbalance_ratio = class_counts / total_pixels
    return class_counts, class_imbalance_ratio

# Compute class distribution for training, validation and test sets
train_class_counts, train_class_imbalance_ratio = compute_class_distribution(y_train, num_classes)
val_class_counts, val_class_imbalance_ratio = compute_class_distribution(y_val, num_classes)
test_class_counts, test_class_imbalance_ratio = compute_class_distribution(y_test, num_classes)

# Print class counts and imbalance ratio
print("Training Set:")
print("Class Counts:", train_class_counts)
print("Class Imbalance Ratio:", train_class_imbalance_ratio)

print("\nValidation Set:")
print("Class Counts:", val_class_counts)
print("Class Imbalance Ratio:", val_class_imbalance_ratio)

print("\nTest Set:")
print("Class Counts:", test_class_counts)
print("Class Imbalance Ratio:", test_class_imbalance_ratio)

Mounted at /content/drive
Image Shape with PCA: (4000, 128, 128, 5)
Label Shape with PCA: (4000, 128, 128)
Number of classes: 5
Train set:
  Image shape: (1800, 128, 128, 5)
  Label shape: (1800, 128, 128)
Validation set:
  Image shape: (600, 128, 128, 5)
  Label shape: (600, 128, 128)
Test set:
  Image shape: (1600, 128, 128, 5)
  Label shape: (1600, 128, 128)
*************************************************************
After Normalization
(1800, 128, 128, 5)
(600, 128, 128, 5)
(1600, 128, 128, 5)
Training Set:
Class Counts: [  203295 19218594  9666862     8551   393898]
Class Imbalance Ratio: [6.89341227e-03 6.51672160e-01 3.27788018e-01 2.89950901e-04
 1.33564589e-02]

Validation Set:
Class Counts: [ 118240 6620763 2957094    3215  131088]
Class Imbalance Ratio: [1.20279948e-02 6.73498840e-01 3.00811157e-01 3.27046712e-04
 1.33349609e-02]

Test Set:
Class Counts: [  366845 17067986  8398056     4382   377131]
Class Imbalance Ratio: [1.39940262e-02 6.51091995e-01 3.20360413e-01 1.67

Development of Model/Architecture to capture Spatial-Spectral Characteristics

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

model_file = "/content/drive/My Drive/model_semantic_segmentation.h5"
early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', # what is the metric to measure
                              patience = 15, # how many epochs to continue running the model after seeing an increase in val_loss
                              restore_best_weights = True) # update the model with the best-seen weights?

reduce_LR = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                              factor = 0.1,
                                              patience = 5) # to make sure early_stop is what stops the model

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = model_file,
                                             monitor = 'val_loss',
                                             save_best_only = True)

callback_list = [reduce_LR, early_stop, checkpoint]

# Define the UNet model architecture
def model(input_shape, num_classes):
    inputs = Input(input_shape)
    # Contracting path
    conv1 = Conv2D(16, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(16, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(32, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(32, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(64, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(128, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)
    #drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(256, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(256, 3, activation='relu', padding='same')(conv5)
    #drop5 = Dropout(0.5)(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)

    conv6 = Conv2D(256, 3, activation='relu', padding='same')(pool5)
    conv6 = Conv2D(256, 3, activation='relu', padding='same')(conv6)
    drop6 = Dropout(0.5)(conv6)
    pool6 = MaxPooling2D(pool_size=(2, 2))(drop6)

    conv7 = Conv2D(256, 3, activation='relu', padding='same')(pool6)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)
    drop7 = Dropout(0.5)(conv7)
    pool7 = MaxPooling2D(pool_size=(2, 2))(drop7)

    # Bottleneck
    conv8 = Conv2D(256, 3, activation='relu', padding='same')(pool7)
    conv8 = Conv2D(256, 3, activation='relu', padding='same')(conv8)
    drop8 = Dropout(0.5)(conv8)

    # Expansive path
    x = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(drop8)
    x = concatenate([x, drop7], axis=3)
    x = Conv2D(256, 3, activation='relu', padding='same')(x)
    x = Conv2D(256, 3, activation='relu', padding='same')(x)

    # Expansive path
    x = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(drop7)
    x = concatenate([x, drop6], axis=3)
    x = Conv2D(256, 3, activation='relu', padding='same')(x)
    x = Conv2D(256, 3, activation='relu', padding='same')(x)

    x = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(x)
    x = concatenate([x, conv5], axis=3)
    x = Conv2D(256, 3, activation='relu', padding='same')(x)
    x = Conv2D(256, 3, activation='relu', padding='same')(x)

    x = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(x)
    x = concatenate([x, conv4], axis=3)
    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = Conv2D(128, 3, activation='relu', padding='same')(x)

    x = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(x)
    x = concatenate([x, conv3], axis=3)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)

    x = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(x)
    x = concatenate([x, conv2], axis=3)
    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = Conv2D(32, 3, activation='relu', padding='same')(x)

    x = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(x)
    x = concatenate([x, conv1], axis=3)
    x = Conv2D(16, 3, activation='relu', padding='same')(x)
    x = Conv2D(16, 3, activation='relu', padding='same')(x)

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(x)

    model = Model(inputs=[inputs], outputs=[outputs])

    return model

input_shape = (128, 128, 5)

# Build the model
model = model(input_shape, num_classes)
model.summary()

# Define custom loss function
def custom_loss(y_true, y_pred):
    class_weights = tf.constant([20.0, 1.0, 1.0, 20.0, 20.0], dtype=tf.float32)  # Adjust weights as needed
    weights = tf.gather(class_weights, tf.cast(y_true, dtype=tf.int32))
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred) * weights
    loss = tf.reduce_mean(loss)
    return loss

#adam = Adam(lr=0.001, decay=1e-06)
#sgd = SGD(learning_rate=0.0001, momentum=0.9, nesterov=False)
opt = RMSprop(learning_rate=0.0001)

# Compile the model
model.compile(optimizer=opt, loss=custom_loss, metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks = callback_list)

# Save the trained model
model.save('/content/drive/My Drive/model_semantic_segmentation.h5')

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  736         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 16  2320        ['conv2d[0][0]']                 
                                )                                                             